# Przetwarzanie raportów resuscytacyjnych za pomocą OpenAI API

## Wstęp 
Celem niniejszej pracy jest zaprojektowanie i implementacja systemu przetwarzania szpitalnych raportów resuscytacyjnych. Praca skupi się na opracowaniu programu umożliwiającego efektowną i dokładną analizę przebiegu wielu interwencji naraz oraz wyciągnięciu wniosków, które mogą poprawić procedury medyczne. 

### Import danych
W pierwszej części dane w postaci pliku JSON zostają zaimportowane.

In [2]:
import json
import os

def import_data(filepath): 
  """
  Imports data from a JSON file and converts it to a string format.

  Parameters:
  filepath (str): Path to the JSON file.

  Returns:
  str: Data from the JSON file as a single string with elements separated by newlines.

  Raises:
  FileNotFoundError: If the file does not exist.
  ValueError: If the file content is not a valid JSON.
  TypeError: If the JSON content is not a list of strings.
  """
  if not os.path.exists(filepath):
      raise FileNotFoundError(f"The file '{filepath}' does not exist.")
  
  try:
      with open(filepath, 'r') as file:
          data = json.load(file)
      if not isinstance(data, list) or not all(isinstance(item, str) for item in data):
          raise TypeError("The JSON content must be a list of strings.")
      return "\n".join(data)
  
  except json.JSONDecodeError as e:
      raise ValueError(f"Failed to decode JSON: {e}")

### Podział raportów na paczki
W tej sekcji raporty zostają podzielone na paczki (batches), tak, aby zoptymalizować liczbę tokenów wysyłanych do modelu. 
Token to najmniejsza jednostka tekstu przetwarzana przez model językowy, taka jak słowo, część słowa, znak interpunkcyjny lub spacja. W przypadku uzywanego tutaj modelu GPT-4-turbo maksymalna liczba tokenów wynosi 4096, stąd zapytanie razem z raportami musi mieścić się w tym zakresie.
Instrukcja wysyłana jest do modelu razem z kolejnymi paczkami az do momentu wyczerpania paczek. 

In [40]:
import tiktoken

def split_reports_into_batches(report_list, prompt, max_tokens=4096):
    """
    Splits a list of reports into batches based on token limits, ensuring that the total token count
    (including a given prompt) does not exceed the specified maximum tokens per batch.

    Parameters:
        report_list (list of str): A list of report strings to be split into batches.
        prompt (str): The prompt text to include with each batch, which affects the token limit.
        max_tokens (int, optional): The maximum number of tokens allowed in a single batch, including the prompt, defaults to 4096.

    Returns:
        list of list of str: A list of batches, where each batch is a list of reports whose combined token count 
        (plus the prompt) stays within the maximum token limit.
    """
    tokenizer = tiktoken.get_encoding("cl100k_base")
    
    prompt_tokens = len(tokenizer.encode(prompt))
    max_tokens_per_batch = max_tokens - prompt_tokens

    batches = []
    current_batch = []
    current_tokens = 0

    for report in report_list:
        report_tokens = tokenizer.encode(report)
        report_token_count = len(report_tokens)

        if current_tokens + report_token_count > max_tokens_per_batch:
            batches.append(current_batch)
            current_batch = []
            current_tokens = 0

        current_batch.append(report)
        current_tokens += report_token_count

    if current_batch:
        batches.append(current_batch)

    return batches

### Połączenie z modelem 
Zachodzi domyślnie dla modelu GPT-4-turbo.

In [4]:
from openai import OpenAI

def get_response(prompt):
    """
    Sends a prompt to an external chat model (gpt-4-turbo) and retrieves the response.

    Parameters:
    prompt (str): The input string to be processed by the model.

    Returns:
    str: The response generated by the model.

    Raises:
    ValueError: If the prompt is empty or invalid.
    Exception: For any unexpected errors during the API call.
    """
    try:
        if not isinstance(prompt, str) or not prompt.strip():
            raise ValueError("Prompt must be a non-empty string.")
        
        response = client.chat.completions.create(
            model="gpt-4-turbo",
            temperature=0.0,
            messages=[
                {"role": "system", "content": "Jesteś pomocnym asystentem."},
                {"role": "user", "content": prompt}
            ],
        )
        
        assistant_response = response.choices[0].message.content
        
        if not assistant_response or not isinstance(assistant_response, str):
            raise ValueError("Model response is empty or invalid.")
        
        return assistant_response.strip()

    except ValueError as ve:
        print(f"Input or response validation error: {ve}")
        raise  
        
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        raise 

### Prompt i przetwarzanie odpowiedzi modelu
Prompt wysyłany do modelu ma dwie funkcje. 

1. Oddzielenie raportów resuscytacyjnych od raportów jedyne wspominających o poprzedniej resucytacji (w tej sytuacji model ma zwrócić łańcuch znaków 'BRAK'). 

2. Zwrócenie przez model odpowiedzi na raport resuscytacyjny w postaci oddzielonych od siebie przecinkiem słów kluczowych (takich jak asystolia, ambu, defibrylacja, ROSC), które mają informować w jak najbardziej skróconej i gotowej do przetworzenia formie o procedurach, jakie miały miejsce podczas resuscytacji. 

Treść wszystkich raportów jest wysoce róznorodna i pojawiają się w nich takze informacje o np. problemach związanych z intubacją, chorobach współtowarzyszących pacjenta lub kilkukrotnych defibrylacjach podczas jednej resuscytacji. W promptcie skupiono się jednak na najczęściej powtarzanych procedurach, ograniczając "słownik" odpowiedzi modelu do konkretnych pojęć. 

W tej instrukcji wykorzystano następujące metody prompt engineering: 
#### Role instruction 
("Jesteś specjalistą od analizy danych medycznych"): Pomaga skoncentrować działanie modelu na odpowiedniej dziedzinie i nadaje kontekst.

#### Explicit instructions
Szczegółowo opisano zadanie podziału raportów na dwie kategorie i podano wytyczne, jak postępować w przypadku kazdej z kategorii. 

#### Constrained output
Zdefiniowano słownik pojęć, aby zredukować ryzyko uzycia nieodpowiednich terminów. 

#### Few-Shot Learning
Dołączono przykłady raportów i ich odpowiedzi. 

#### Standardized Output 
Wymóg utrzymania standardowego formatu odpowiedzi w mianowniku (np. “czas X”, “adrenalina”).

#### Disambiguation 
Rozwiązano potencjalne niejasności poprzez wyjaśnienie wyjątków (np. bradyasystolia → asystolia) oraz oczekiwana reakcja na brak danych (“BRAK”) została zdefiniowana jednoznacznie.

#### Task Prioritization
Zadanie jest podzielone na dwa logiczne kroki:
	1.	Przyporządkowanie raportu do kategorii.
	2.	Generowanie odpowiedzi w formie słów kluczowych lub zwrócenie “BRAK”.

#### Highlighting Key Points
Uzycie pogrubienia ("**Kategoria 1**") i dodatkowych tagów (<\raporty>) ułatwiające modelowi priorytetyzację.

In [4]:
prompt_summarize = """
Jesteś specjalistą od analizy danych medycznych. Przeczytaj podane raporty (konsultacje) szpitalne i przyporządkuj kazdy do jednej z dwóch kategorii:
1. **Kategoria 1**: Konsultacja opisuje aktywną resuscytację pacjenta, obejmującą działania takie jak: podjęcie resuscytacji, intubacja, podanie leków ratunkowych (np. adrenaliny), użycie defibrylatora, wykonywanie masażu serca itp.
2. **Kategoria 2**: Konsultacja odnosi się do wcześniejszej resuscytacji, ale nie opisuje aktywnej resuscytacji w trakcie danej konsultacji (np. stan pacjenta po wcześniejszym NZK, informacje o przebytej resuscytacji bez szczegółów interwencji w obecnym czasie).

Twoje zadanie:
1. Jeśli raport należy do **Kategorii 1**, zamień raport na zestawienie w formie słów kluczowych. Słowa kluczowe mają być zapisane w mianowniku i w pełni standaryzowane. Oto słowa kluczowe, z których masz korzystać w odpowiedzi i z niczego innego:
   - **Mechanizmy**: "asystolia" (bradyasystolię tez traktuj jako asystolię), "VF" (migotanie komór), "PEA" (aktywność elektryczna bez tętna).
   - **Czas**: czas trwania resuscytacji w minutach, określony jako różnica czasu zakończenia resuscytacji i jej rozpoczęcia lub wystąpienia NZK ("czas X"). Jeśli brak informacji, pomiń parametr czasu.
   - **Procedury**: "intubacja", "lucas", "ambu", "masaż serca", "defibrylacja", podane leki: "adrenalina", "bikarbonat", "amiodaron", "inne leki".
   - **Wynik**: "ROSC", "zgon".

   Jeśli w raporcie brakuje danych, pomiń te parametry w odpowiedzi. Zachowaj standardowy format wyjściowy.

2. Jeśli raport należy do **Kategorii 2**, zwróć dokładnie tekst: 'BRAK'.

### Przykłady
**Raporty:**  
["Wezwanie na interwencję z powodu nagłego zatrzymania krążenia (NZK). Według relacji personelu SOR, zatrzymanie krążenia nastąpiło około godziny 14:30 w mechanizmie asystolii. Natychmiast rozpoczęto RK, wykonując masaż zewnętrzny serca oraz intubując pacjenta przez usta. Rozpoczęto wentylację przy pomocy worka Ambu, a masaż serca kontynuowano przy użyciu urządzenia Lucas.
W trakcie działań podano łącznie 6 mg adrenaliny. Pomimo przeprowadzonych działań resuscytacyjnych, nie uzyskano ROSC. O godzinie 15:00 stwierdzono zgon pacjenta.", "Przy przybyciu pacjent był masowany oraz wentylowany za pomocą worka Ambu. Pacjenta zaintubowano przez usta rurką 8.5. Podawano adrenalinę zgodnie z aktualnymi wytycznymi. Pomimo prowadzonej akcji resuscytacyjnej nie uzyskano ROSC. O godzinie 16:10 stwierdzono zgon chorego.", "Pacjent NN o nieznanej przeszłości chorobowej, zaniedbany higienicznie. Stan po NZK i skutecznej resuscytacji wykonanej przez ZRM. Obecnie pacjent w stanie śpiączki, wentylowany mechanicznie, hemodynamicznie stabilny na wlewie noradrenaliny. W wykonanym TK głowy wykluczono krwawienie. Zalecenia: pełne monitorowanie, kontrolne badania laboratoryjne, RTG klatki piersiowej, ewentualna decyzja o koronarografii po ocenie stanu neurologicznego."]
**Odpowiedź:**  
["asystolia, masaż serca, intubacja, ambu, lucas, adrenalina, czas 30, zgon", "masaż serca, intubacja, ambu, adrenalina, zgon", "BRAK"]
""" 

def summarize_batches(batches):
    """
    Processes batches of medical reports and combines their summarized results.

    Parameters:
    batches (list of list of str): A list of batches, where each batch is a list of medical reports.

    Returns:
    list of str: Combined list of summarized reports without leading numbers or extra quotes.
    """
    combined_summaries = []

    for batch in batches:
        try:
            batch_content = "\n\n".join(batch)
            final_prompt = prompt_summarize + f"\n\n<raporty>\n{batch_content}\n</raporty>"
            
            response = get_response(final_prompt)
            
            summaries = [item.strip() for item in response.split("\n") if item.strip()]
            
            clean_summaries = []
            for summary in summaries:
                clean_summary = summary.split(".", 1)[-1].strip()
                clean_summary = clean_summary.strip('"')
                clean_summaries.append(clean_summary)
            
            combined_summaries.extend(clean_summaries)
        
        except Exception as e:
            print(f"Error processing batch: {batch}\nError: {e}")

    return combined_summaries

In [74]:
def write_file(str_data, fname="summ_data.txt"):
    """
    Checks if a specified file exists. If the file does not exist, it generates summarized data 
    using the provided summarize_func and writes it to the file.

    Parameters:
    fname (str): The name of the file to check or create, default is "summ_data.txt".

    Returns:
    None
    """
    try:
        if os.path.isfile(fname):
            print(f"File '{fname}' already exists. No action taken.")
            return

        with open(fname, "w") as f:
            f.write(str_data)

    except FileNotFoundError as fnf_error:
        print(f"File not found error: {fnf_error}")
        raise  

    except ValueError as ve:
        print(f"Value error: {ve}")
        raise

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        raise

### Import danych do biblioteki pandas i ich przetwarzanie


In [39]:
import pandas as pd
import ast 

with open('summ_data.txt', 'r', encoding='UTF-8') as file:
    data_as_string = file.read() 

data_list = ast.literal_eval(data_as_string)
data_list[0:5]

['asystolia, masaż serca, intubacja, ambu, lucas, adrenalina, czas 30, zgon',
 'BRAK',
 'asystolia, masaż serca, intubacja, ambu, adrenalina, zgon',
 'VF, masaż serca, intubacja, adrenalina, amiodaron, defibrylacja, inne leki, ROSC',
 'asystolia, masaż serca, intubacja, ambu, adrenalina, zgon']

In [14]:
df = pd.DataFrame(data_list, columns=["report"])
df                

,report
0,"asystolia, masaż serca, intubacja, ambu, lucas..."
1,BRAK
2,"asystolia, masaż serca, intubacja, ambu, adren..."
3,"VF, masaż serca, intubacja, adrenalina, amioda..."
4,"asystolia, masaż serca, intubacja, ambu, adren..."
...,...
331,"PEA, masaż serca, intubacja, lucas, adrenalina..."
332,"masaż serca, intubacja, ambu, lucas, adrenalin..."
333,"masaż serca, intubacja, adrenalina, zgon"
334,"asystolia, masaż serca, adrenalina, czas 20, zgon"


In [37]:
# Pozbycie się konsultacji wspominających o przeszłej resuscytacji 
df = df[df['report'] != 'BRAK']

df.reset_index(drop=True, inplace=True)

def process_report(report):
    # Kategorie
    mechanisms = {"asystolia", "VF", "PEA"}
    procedures = {"intubacja", "lucas", "ambu", "masaż serca", "defibrylacja",
                  "adrenalina", "bikarbonat", "amiodaron", "inne leki"}
    results = {"ROSC", "zgon"}
    
    # Parsowanie raportu
    elements = set(report.split(", "))
    mech = sorted(list(elements & mechanisms))  # Posegregowane mechanizmy
    proc = sorted(list(elements & procedures))  # Wszystkie procedury
    result = sorted(list(elements & results))  # Wynik 
    
    # Znalezienie czasu
    time = [el for el in elements if el.startswith("czas")]
    
    # Obsługa czasu
    extracted_time = None
    if time:
        try:
            time_value = time[0].split()[1]
            if "-" in time_value:  # Obsługa zakresów czasu (np. "czas 20-30")
                lower, upper = map(int, time_value.split("-"))
                extracted_time = (lower + upper) / 2  # Średnia z zakresu
            else:
                extracted_time = int(time_value)  
        except ValueError:
            extracted_time = None  # Jeśli nie jest liczbą ("czas X"), ustaw na None
    
    return {
        "mechanisms": mech if mech else None,  
        "procedures": proc,                    
        "result": result if result else None,  
        "time": extracted_time               
    }

processed_data = df["report"].apply(process_report)

df_processed = pd.DataFrame(processed_data.tolist())
df_processed

,mechanisms,procedures,result,time
0,[asystolia],"[adrenalina, ambu, intubacja, lucas, masaż serca]",[zgon],30.0
1,[asystolia],"[adrenalina, ambu, intubacja, masaż serca]",[zgon],NaN
2,[VF],"[adrenalina, amiodaron, defibrylacja, inne lek...",[ROSC],NaN
3,[asystolia],"[adrenalina, ambu, intubacja, masaż serca]",[zgon],NaN
4,[asystolia],"[adrenalina, bikarbonat, inne leki, intubacja,...",[ROSC],8.0
...,...,...,...,...
266,[asystolia],[intubacja],[zgon],NaN
267,[PEA],"[adrenalina, intubacja, lucas, masaż serca]",[zgon],NaN
268,None,"[adrenalina, ambu, intubacja, lucas, masaż serca]",[ROSC],10.0
269,None,"[adrenalina, intubacja, masaż serca]",[zgon],NaN


In [33]:
key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=key)
filepath = os.getenv("filepath")